In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoModel,
    BitsAndBytesConfig,
    TrainingArguments,
    TextIteratorStreamer
)

/scratch/users/anup/miniconda3/envs/finetune-gllm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-03-26 12:04:03,221] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
# https://huggingface.co/docs/peft/main/en/tutorial/peft_model_config

import deepspeed
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModel
import torch
from peft import AutoPeftModelForCausalLM, PeftModel


base_dir = "llama-linear-layers-all-conv-Feb-19-24-1"
checkpoint_number = "checkpoint-2500"
checkpoint_path = "{}/{}".format(base_dir, checkpoint_number)
model_name = "NousResearch/Llama-2-7b-chat-hf"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype
)

original_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    use_cache=False,
    device_map="auto"
)

peft_model = PeftModel.from_pretrained(original_model, checkpoint_path)
#peft_model = peft_model.merge_and_unload()
restored_tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.19s/it]
/scratch/users/anup/miniconda3/envs/finetune-gllm/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/scratch/users/anup/miniconda3/envs/finetune-gllm/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_samp

In [7]:
# bitsandbytes from 0.41.1 to 0.43.0
# transformers from 4.36.2 to 4.37.0
peft_model.save_pretrained("saved_model_hub/fine-tuned-gllm") #saved_model_hub/fine-tuned-gllm

In [8]:
peft_model.push_to_hub("anuprulez/fine-tuned-gllm")

adapter_model.safetensors: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 649M/649M [00:25<00:00, 25.7MB/s]


CommitInfo(commit_url='https://huggingface.co/anuprulez/fine-tuned-gllm/commit/c5a3ec244a88e23f64089cd231fba454b6d3df4f', commit_message='Upload model', commit_description='', oid='c5a3ec244a88e23f64089cd231fba454b6d3df4f', pr_url=None, pr_revision=None, pr_num=None)